In [1]:
from __future__ import division
import colorsys
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
import numpy as np
from frostsynth.waveform import *
from frostsynth.sampling import *
from frostsynth.ipython import *
output_notebook()

Loading BokehJS ...

In [2]:
N = 15
X = 10
S = 13

In [3]:
x = np.arange(0, 1 << N) / (1 << N)

In [4]:
pw = figure(title="waveform", plot_height=200, plot_width=800, y_range=(-1, 1))
rw = pw.line(x[:1<<X], x[:1<<X])
ps = figure(title="spectrum", plot_height=200, plot_width=800, y_range=(0, 1.2))
rs = ps.vbar(np.arange(1<<S), bottom=0, width=20, top=np.zeros(1<<S), color=["#abcdef"] * (1 << S))

In [5]:
def update_plot(y):
    rw.data_source.data["y"] = y[:1<<X]
    s = (np.fft.rfft(y) / (1 << S))[:1 << S]
    angle = (np.angle(s) + np.pi) / (2 * np.pi)
    rgb = np.array([colorsys.hls_to_rgb(a, 0.4, 0.8) for a in angle]) * 255
    color = ["#%02x%02x%02x" % tuple(c) for c in rgb]
    rs.data_source.data["top"] = abs(s)
    rs.data_source.data["fill_color"] = color
    rs.data_source.data["line_color"] = color
    push_notebook()

In [6]:
def update_fm(amplitude, index_c, index_m, offset, carrier, modulator):
    y = carrier(index_c * x + amplitude * modulator(index_m * x) + offset)
    update_plot(y)

In [7]:
show(column(pw, ps), notebook_handle=True);

In [8]:
waveforms = {"sin": sine, "saw": saw, "par": par, "cub": cub, "tooth": tooth, "tang": tang}
interact(update_fm, amplitude=(0, 4, 0.05), index_c=(0, 1000), index_m=(1, 1000), offset=(0, 1, 0.05), carrier=waveforms, modulator=waveforms);

interactive(children=(FloatSlider(value=2.0, description=u'amplitude', max=4.0, step=0.05), IntSlider(value=500, description=u'index_c', max=1000), IntSlider(value=500, description=u'index_m', max=1000, min=1), FloatSlider(value=0.0, description=u'offset', max=1.0, step=0.05), Dropdown(description=u'carrier', options={'par': <function par at 0x7f084d75b7d0>, 'tang': <function tang at 0x7f084d75b140>, 'tooth': <function tooth at 0x7f084d75b230>, 'cub': <function cub at 0x7f084d75b8c0>, 'saw': <function saw at 0x7f084d75b758>, 'sin': <function sine at 0x7f084d7c3668>}, value=<function par at 0x7f084d75b7d0>), Dropdown(description=u'modulator', options={'par': <function par at 0x7f084d75b7d0>, 'tang': <function tang at 0x7f084d75b140>, 'tooth': <function tooth at 0x7f084d75b230>, 'cub': <function cub at 0x7f084d75b8c0>, 'saw': <function saw at 0x7f084d75b758>, 'sin': <function sine at 0x7f084d7c3668>}, value=<function par at 0x7f084d75b7d0>), Output()), _dom_classes=('widget-interact',))

In [9]:
set_sample_rate(44100)

In [10]:
t = trange(3)
p = 2 * t

In [12]:
Audio(0.1 * sine(212 * p + cub(911 * p) * (0.5*np.exp(-t*4) + 0.1)) * np.exp(-2*t))